In [1]:
import numpy as np
import scipy
import keras
from keras.models import Sequential, Model
from keras.layers import Dense, Dropout, Flatten, Input, Embedding
from keras.layers import Conv2D, MaxPooling2D, AveragePooling2D
import tensorflow as tf
print(tf.__version__)
print( keras.__version__)
import h5py

C:\Users\neal\Anaconda3\envs\keras\lib\site-packages\h5py\__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


1.5.0
2.1.3


In [2]:
def getTest(whitewins, blackwins):
    test = []
    test_l = []
    for i in range(whitewins.shape[0]):
        if np.random.randn(1)>0.5:
            test.append([whitewins[i],blackwins[i]])
            test_l.append([1, 0])
        else:
            test.append([blackwins[i],whitewins[i]])
            test_l.append([0, 1])
#         temp_test = np.zeros((len(test), 2, input_size))
#     for i in range(len(test)):
#         first = bitifyFEN(test[i][0])	
#         second = bitifyFEN(test[i][1])	
#         elem = [first,second]
#         temp_test[i] = elem
    return (np.array(test), np.array(test_l))

def getBatchAE(start, size):
    global whiteWins 
    global blackWins 
    size = size//2
    start = start*size

    temp1 = np.concatenate((whiteWins[start:start+size],blackWins[start:start+size]),axis=0)
    np.random.shuffle(temp1)
    return temp1

ff = h5py.File("raw_chess_data.hdf5", "r")

# print(ff.keys())
whiteWins2 = ff['white']
blackWins2 = ff['black']
validation_test, validation_test_l = getTest(whiteWins2[:100000],blackWins2[:100000])
print(whiteWins2.shape)
print(blackWins2.shape)

(2598560, 769)
(1925000, 769)


In [3]:
def getBatchMLP(start, size):
    import random
    global whiteWins 
    global blackWins 

    xR = []
    lR = []

    for i in range(start,start+size):
        if random.random() > 0.5:
            elem = [whiteWins[i], blackWins[i]]
            elem_l = [1,0]
        else:
            elem = [blackWins[i], whiteWins[i]]
            elem_l = [0,1]
        xR.append(elem)
        lR.append(elem_l)
    return (np.array(xR), np.array(lR))

In [37]:
encoding_dim2 = 400

encoded2 = Dense(encoding_dim2, activation='selu')(encoded)
decoded2 = Dense(encoding_dim, activation='sigmoid')(encoded2)
decoded_1 = autoencoder1.layers[-1](decoded2)
autoencoder2 = Model(input_img, decoded_1)
autoencoder2.compile(optimizer='adadelta', loss='binary_crossentropy')

In [ ]:
encoding_dim3 = 200

encoded3 = Dense(encoding_dim3, activation='selu')(encoded2)
decoded3 = Dense(encoding_dim2, activation='sigmoid')(encoded3)
decoded_1 = autoencoder2.layers[-1](decoded2)

In [41]:
# do them all at once:
dims = [600,400,200,100]
input_img = Input(shape=(769,))
encoded1 = Dense(dims[0], activation='selu')(input_img)
encoded2 = Dense(dims[1], activation='selu')(encoded1)
encoded3 = Dense(dims[2], activation='selu')(encoded2)
encoded4 = Dense(dims[3], activation='selu')(encoded3)
decoded4 = Dense(dims[2], activation='selu')(encoded4)
decoded3 = Dense(dims[1], activation='selu')(decoded4)
decoded2 = Dense(dims[0], activation='selu')(decoded3)
decoded1 = Dense(769, activation='softmax')(decoded2)
autoencoder = Model(input_img, decoded1)
autoencoder.compile(optimizer='adadelta', loss='categorical_crossentropy')

In [26]:
whiteWins = np.random.permutation(whiteWins2[100000:])[:1000000]
blackWins = np.random.permutation(blackWins2[100000:])[:1000000]
TOTAL_AE = 1000000
BS_AE = 25000
EPOCHS_AE = 400
total_batch = int(TOTAL_AE/BS_AE)

In [32]:
# print(x_train.shape)

(25000, 769)


In [42]:
for j in range(EPOCHS_AE):
    for i in range(total_batch):
        x_train = getBatchAE(i, BS_AE)
        autoencoder.fit(x_train, x_train,
                        epochs=1,
                        batch_size=128,
                        shuffle=True)#,
    #                    validation_data = (x_train, x_train))

Epoch 1/1
25000/25000 [==============================] - 3s 115us/step - loss: 94.6104
Epoch 1/1
25000/25000 [==============================] - 2s 80us/step - loss: 84.2459
Epoch 1/1
25000/25000 [==============================] - 2s 79us/step - loss: 81.3506
Epoch 1/1
25000/25000 [==============================] - 2s 75us/step - loss: 80.2827
Epoch 1/1
25000/25000 [==============================] - 2s 75us/step - loss: 79.4486
Epoch 1/1
25000/25000 [==============================] - 2s 76us/step - loss: 78.9724
Epoch 1/1
25000/25000 [==============================] - 2s 75us/step - loss: 78.2061
Epoch 1/1
25000/25000 [==============================] - 2s 72us/step - loss: 78.1314
Epoch 1/1
25000/25000 [==============================] - 2s 73us/step - loss: 77.8237
Epoch 1/1
25000/25000 [==============================] - 2s 77us/step - loss: 77.2349
Epoch 1/1
25000/25000 [==============================] - 2s 80us/step - loss: 76.7317
Epoch 1/1
25000/25000 [==============================

KeyboardInterrupt: 

In [ ]:
#     #______________________________________________________ link definition
#     input_to_link = Input((1, img_rows, img_cols))
#     ...several convolutions...
#     link_model = Model(input=input_to_link, output=conv4)   # a chain is made of links

#     #______________________________________________________ chain definition
#     input_to_chain = Input((1, img_rows, img_cols))
#     output_link_1 = link_model(input_to_chain)
#     output_link_2 = link_model(output_link_1)
#     chain_model = Model(input=input_to_chain, output=output_link_2)

In [47]:
link_model = Model(input_img, encoded4)
input_1 = Input(shape=(769,))
input_2 = Input(shape=(769,))
output_link_1 = link_model(input_1)
output_link_2 = link_model(input_2)


In [4]:
# Try it without training the two autoencoders first:
input_img = (769,)
dims = [600, 400, 200, 100]
auto1 = Sequential()
auto1.add(Dense(600, activation='selu',input_shape=input_img))
for i in range(1,len(dims)):
    auto1.add(Dense(dims[i], activation='selu'))


In [19]:
main_input = Input(shape=(769,), dtype='int32', name='main_input')
x = Embedding(output_dim=600, input_dim=769)(main_input)
# This embedding layer will encode the input sequence
# into a sequence of dense 512-dimensional vectors.
# x = Dense(600, activation='selu', input_shape = (769,))
x = Dense(400, activation='selu')(x)
x = Dense(200, activation='selu')(x)
encode_out = Dense(100, activation='selu')(x)

main_input2 = Input(shape=(769,), dtype='int32', name='main_input2')
x = Embedding(output_dim=600, input_dim=769)(main_input2)
# This embedding layer will encode the input sequence
# into a sequence of dense 512-dimensional vectors.
# x = Dense(600, activation='selu', input_shape = (769,))
x = Dense(400, activation='selu')(x)
x = Dense(200, activation='selu')(x)
encode_out2 = Dense(100, activation='selu')(x)
# A LSTM will transform the vector sequence into a single vector,
# containing information about the entire sequence
# lstm_out = LSTM(32)(x)

In [51]:
# create the rest of the model
# fc_dims = [200, 400, 200, 100]
# fully_connected = Sequential()
# fully_connected.add(Dense(fc_dims[i], activation='selu', input_shape = (200,)))
# for i in range(1, len(fc_dims)):
#     fully_connected.add(Dense(fc_dims[i], activation='selu'))
# model = Model([auto1, auto1], fully_connected)
x = keras.layers.concatenate([output_link_1, output_link_2])
# x = Flatten()(x)
x = Dense(400, activation='relu')(x)
x = Dense(200, activation='relu')(x)
x = Dense(100, activation='relu')(x)
main_output = Dense(2,  activation='softmax', name='main_output')(x)
model = Model(inputs=[input_1, input_2], outputs=[main_output])
model.compile(optimizer='adadelta', loss='categorical_crossentropy',  metrics=['acc'])

In [21]:
print(main_output)

Tensor("main_output_4/Softmax:0", shape=(?, 2), dtype=float32)


In [53]:
EPOCHS_MLP = 1000
TOTAL_MLP = 1000000
BS_MLP = 25000
total_batch = int(TOTAL_MLP/BS_MLP)
for j in range(EPOCHS_MLP):
    print(j)
    whiteWins = np.random.permutation(whiteWins2[100000:])[:1000000]
    blackWins = np.random.permutation(blackWins2[100000:])[:1000000]

    for i in range(total_batch): 
        batch_xs, batch_ys = getBatchMLP(i*BS_MLP, BS_MLP)
        model.fit([batch_xs[:,0,:],batch_xs[:,1,:]], [batch_ys],
                        epochs=1,
                        verbose=0,
                        batch_size=128,
                        shuffle=True)#,
#                         validation_data = ([validation_test[:,0,:], validation_test[:,1,:]], [validation_test_l]))
    if j % 3 == 0:
        score = model.evaluate([validation_test[:,0,:], validation_test[:,1,:]], [validation_test_l], verbose=0)
        print('Test loss:', score[0])
        print('Test accuracy:', score[1])

0
Test loss: 0.34942051426410675
Test accuracy: 0.82233
1
2
3
Test loss: 0.3405716425162554
Test accuracy: 0.83292
4
5
6
Test loss: 0.3742014820575714
Test accuracy: 0.81911
7
8
9
Test loss: 0.3544433444869518
Test accuracy: 0.81185
10
11
12
Test loss: 0.34497075414180756
Test accuracy: 0.82927
13
14
15
Test loss: 0.3963639073586464
Test accuracy: 0.8077
16
17
18
Test loss: 0.3793555604600906
Test accuracy: 0.81953
19
20
21
Test loss: 0.3812181893348694
Test accuracy: 0.81862
22
23
24
Test loss: 0.40839160892009735
Test accuracy: 0.79608
25
26
27
Test loss: 0.39549178974628446
Test accuracy: 0.81541
28
29
30
Test loss: 0.3835984764158726
Test accuracy: 0.80804
31
32


KeyboardInterrupt: 

In [ ]:
# try to create the encoders separately
